In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
import re
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from nltk.stem import *
import copy
#nltk.download()

In [2]:
testset = pd.read_csv('test.csv')
trainset = pd.read_csv('train.csv')
sampleset = pd.read_csv('sample.csv')

In [3]:
trainset.info()
trainset.head()
trainset[:20]
#trainset.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1697533 entries, 0 to 1697532
Data columns (total 9 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Id                      int64  
 1   ProductId               object 
 2   UserId                  object 
 3   HelpfulnessNumerator    int64  
 4   HelpfulnessDenominator  int64  
 5   Score                   float64
 6   Time                    int64  
 7   Summary                 object 
 8   Text                    object 
dtypes: float64(1), int64(4), object(4)
memory usage: 116.6+ MB


,Id,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,0,0005019281,ADZPIG9QOCDG5,0,0,4.0,1203984000,good version of a classic,This is a charming version of the classic Dick...
1,1,0005019281,A35947ZP82G7JH,0,0,3.0,1388361600,Good but not as moving,It was good but not as emotionally moving as t...
2,2,0005019281,A3UORV8A9D5L2E,0,0,3.0,1388361600,Winkler's Performance was ok at best!,"Don't get me wrong, Winkler is a wonderful cha..."
3,3,0005019281,A1VKW06X1O2X7V,0,0,5.0,1202860800,It's an enjoyable twist on the classic story,Henry Winkler is very good in this twist on th...
4,4,0005019281,A3R27T4HADWFFJ,0,0,4.0,1387670400,Best Scrooge yet,This is one of the best Scrooge movies out. H...
5,5,0005019281,A2L0G56BNOTX6S,0,0,NaN,1383696000,Dickens updated.,This has been a favorite movie of mine for a l...
6,6,0005019281,A5NYUBEKXFLX5,1,1,5.0,1230595200,A MUST-HAVE FOR ANY VIDEO CHRISTMAS COLLECTION!!,This is the American adaptation of the Charles...
7,7,0005019281,A2DJ8B8GE4V2VD,0,0,5.0,1260835200,An American Christmas Carol,Glad that this american classic came out on dv...
8,8,0005019281,AWF2S3UNW9UA0,0,0,5.0,1386201600,an american christmas carol,A good Christmas carol dhenry winkler one duri...
9,9,0005019281,A3O4UUT83DG3OU,0,0,5.0,1379721600,Fantastic!,How a bitter old man comes to know the true me...


# Filling missing value with text and summary

In [4]:
trainset['Text'] = np.where(trainset['Text'].isnull(), trainset['Summary'], trainset['Text'])
trainset['Summary'] = np.where(trainset['Summary'].isnull(), trainset['Text'], trainset['Summary'])
trainset.isna().sum()

Id                             0
ProductId                      0
UserId                         0
HelpfulnessNumerator           0
HelpfulnessDenominator         0
Score                     300000
Time                           0
Summary                        0
Text                           0
dtype: int64

In [5]:
trainset['ProductId'] = trainset['ProductId'].astype(str)
trainset['ProductId'] = pd.Categorical((pd.factorize(trainset.ProductId)[0] + 1))
trainset[:10]

,Id,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,0,1,ADZPIG9QOCDG5,0,0,4.0,1203984000,good version of a classic,This is a charming version of the classic Dick...
1,1,1,A35947ZP82G7JH,0,0,3.0,1388361600,Good but not as moving,It was good but not as emotionally moving as t...
2,2,1,A3UORV8A9D5L2E,0,0,3.0,1388361600,Winkler's Performance was ok at best!,"Don't get me wrong, Winkler is a wonderful cha..."
3,3,1,A1VKW06X1O2X7V,0,0,5.0,1202860800,It's an enjoyable twist on the classic story,Henry Winkler is very good in this twist on th...
4,4,1,A3R27T4HADWFFJ,0,0,4.0,1387670400,Best Scrooge yet,This is one of the best Scrooge movies out. H...
5,5,1,A2L0G56BNOTX6S,0,0,NaN,1383696000,Dickens updated.,This has been a favorite movie of mine for a l...
6,6,1,A5NYUBEKXFLX5,1,1,5.0,1230595200,A MUST-HAVE FOR ANY VIDEO CHRISTMAS COLLECTION!!,This is the American adaptation of the Charles...
7,7,1,A2DJ8B8GE4V2VD,0,0,5.0,1260835200,An American Christmas Carol,Glad that this american classic came out on dv...
8,8,1,AWF2S3UNW9UA0,0,0,5.0,1386201600,an american christmas carol,A good Christmas carol dhenry winkler one duri...
9,9,1,A3O4UUT83DG3OU,0,0,5.0,1379721600,Fantastic!,How a bitter old man comes to know the true me...


In [6]:
trainset['UserId'] = trainset['UserId'].astype(str)
trainset['UserId'] = pd.Categorical((pd.factorize(trainset.UserId)[0] + 1))
trainset[:10]

,Id,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,0,1,1,0,0,4.0,1203984000,good version of a classic,This is a charming version of the classic Dick...
1,1,1,2,0,0,3.0,1388361600,Good but not as moving,It was good but not as emotionally moving as t...
2,2,1,3,0,0,3.0,1388361600,Winkler's Performance was ok at best!,"Don't get me wrong, Winkler is a wonderful cha..."
3,3,1,4,0,0,5.0,1202860800,It's an enjoyable twist on the classic story,Henry Winkler is very good in this twist on th...
4,4,1,5,0,0,4.0,1387670400,Best Scrooge yet,This is one of the best Scrooge movies out. H...
5,5,1,6,0,0,NaN,1383696000,Dickens updated.,This has been a favorite movie of mine for a l...
6,6,1,7,1,1,5.0,1230595200,A MUST-HAVE FOR ANY VIDEO CHRISTMAS COLLECTION!!,This is the American adaptation of the Charles...
7,7,1,8,0,0,5.0,1260835200,An American Christmas Carol,Glad that this american classic came out on dv...
8,8,1,9,0,0,5.0,1386201600,an american christmas carol,A good Christmas carol dhenry winkler one duri...
9,9,1,10,0,0,5.0,1379721600,Fantastic!,How a bitter old man comes to know the true me...


In [7]:
nunique1 = trainset['ProductId'].nunique()
print(nunique1 )
nunique2 = trainset['UserId'].nunique()
print(nunique2)

50052
123960


# Data Preprocessing

In [8]:
summaryset = trainset.loc[:, ('Id', 'Summary')]
null_index_summaryset = summaryset.loc[summaryset['Summary'].isnull(), :].index
summaryempty = summaryset.iloc[null_index_summaryset]
#print(summaryempty)
#not_index_summaryset = summaryset.loc[summaryset['Summary'].notnull(), :].index
summaryfull1 = summaryset.dropna()
summaryfull = copy.deepcopy(summaryfull1)
summaryfull.reset_index(inplace=True, drop=True)
#print(summaryfull.dtypes)
#print(summaryfull.loc[11603])


textset = trainset.loc[:, ('Id', 'Text')]
null_index_textset = textset.loc[textset['Text'].isnull(), :].index
textempty = textset.iloc[null_index_textset]
#print(textempty)
#not_index_textset = textset.loc[textset['Text'].notnull(), :].index
textfull1 = textset.dropna()
textfull = copy.deepcopy(textfull1)
textfull.reset_index(inplace=True, drop=True)
#print(textfull.dtypes)
#print(textfull.loc[11603])

In [98]:
# Numbers removing
import re
p = re.compile(r'\d+')
#summaryfull['Summary'] = [p.sub('', x) for x in summaryfull['Summary'].tolist()]
textfull['Text'] = [p.sub('', x) for x in textfull['Text'].tolist()]

In [9]:
textfull.head()

,Id,Summary
0,0,good version of a classic
1,1,Good but not as moving
2,2,Winkler's Performance was ok at best!
3,3,It's an enjoyable twist on the classic story
4,4,Best Scrooge yet


In [10]:
#Lowercase and split
import re
p = re.compile(r'[^\w\s]+')
#summaryfull['Summary'] = [p.sub('', x) for x in summaryfull['Summary'].tolist()]
#summaryfull['Summary'] = summaryfull['Summary'].str.lower().str.split()
textfull['Text'] = [p.sub('', x) for x in textfull['Text'].tolist()]
textfull['Text'] = textfull['Text'].str.lower().str.split()

In [11]:
textfull.head()

,Id,Summary
0,0,"[good, version, of, a, classic]"
1,1,"[good, but, not, as, moving]"
2,2,"[winklers, performance, was, ok, at, best]"
3,3,"[its, an, enjoyable, twist, on, the, classic, ..."
4,4,"[best, scrooge, yet]"


In [12]:
#Delete stop words
stop_words = set(stopwords.words('english'))
#summaryfull['Summary'] = summaryfull['Summary'].apply(lambda x: [item for item in x if item not in stop_words])
textfull['Text'] = textfull['Text'].apply(lambda x: [item for item in x if item not in stop_words])

In [13]:
textfull.head()

,Id,Summary
0,0,"[good, version, classic]"
1,1,"[good, moving]"
2,2,"[winklers, performance, ok, best]"
3,3,"[enjoyable, twist, classic, story]"
4,4,"[best, scrooge, yet]"


In [15]:
#Stemming
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
stemmer= PorterStemmer()
#summaryfull['Summary'] = summaryfull['Summary'].apply(lambda x: [stemmer.stem(y) for y in x])
textfull['Text'] = textfull['Text'].apply(lambda x: [stemmer.stem(y) for y in x])

In [16]:
textfull.head()

,Id,Summary
0,0,"[good, version, classic]"
1,1,"[good, move]"
2,2,"[winkler, perform, ok, best]"
3,3,"[enjoy, twist, classic, stori]"
4,4,"[best, scroog, yet]"


In [17]:
#summaryfull['Summary'] = summaryfull['Summary'].apply(', '.join)
textfull['Text'] = textfull['Text'].apply(', '.join)

In [18]:
textfull.head()

,Id,Summary
0,0,"good, version, classic"
1,1,"good, move"
2,2,"winkler, perform, ok, best"
3,3,"enjoy, twist, classic, stori"
4,4,"best, scroog, yet"


In [19]:
p1 = re.compile(r'[^\w\s]+')
#summaryfull['Summary'] = [p1.sub('', x) for x in summaryfull['Summary'].tolist()]
#summaryfull['Summary'] = summaryfull['Summary'].str.lower()
textfull['Text'] = [p1.sub('', x) for x in textfull['Text'].tolist()]
textfull['Text'] = textfull['Text'].str.lower()

In [20]:
textfull.head()

,Id,Summary
0,0,good version classic
1,1,good move
2,2,winkler perform ok best
3,3,enjoy twist classic stori
4,4,best scroog yet


In [6]:
#textfull.to_csv('textfull.csv')

# Train Model

In [62]:
textfull = pd.read_csv('textfull.csv')

In [63]:
summaryfull = pd.read_csv('summaryfull.csv')

In [64]:
textfull=pd.concat([textfull,trainset.loc[:, ('UserId', 'ProductId','HelpfulnessNumerator','HelpfulnessDenominator', 'Score')]],axis=1)

In [65]:
textfull.info()
textfull.isna().sum()
null_index_textfull = textfull.loc[textfull['Text'].isnull(), :].index
textfullempty = textfull.iloc[null_index_textfull]
print(textfullempty)
print(textfull.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1697533 entries, 0 to 1697532
Data columns (total 9 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   Unnamed: 0              int64   
 1   Id                      int64   
 2   Text                    object  
 3   UserId                  category
 4   ProductId               category
 5   HelpfulnessNumerator    int64   
 6   HelpfulnessDenominator  int64   
 7   Time                    int64   
 8   Score                   float64 
dtypes: category(2), float64(1), int64(5), object(1)
memory usage: 111.2+ MB
         Unnamed: 0       Id Text  UserId ProductId  HelpfulnessNumerator  \
9208           9208     9208  NaN    1212       155                     0   
55514         55514    55514  NaN   28824       801                     0   
123050       123050   123050  NaN   43707      1486                     0   
209778       209778   209778  NaN   66979      3124                     0   
300091  

In [66]:
textfull.dropna(subset = ['Text'],inplace = True)
textfull.reset_index(inplace=True, drop=True)
textfull.isna().sum()

Unnamed: 0                     0
Id                             0
Text                           0
UserId                         0
ProductId                      0
HelpfulnessNumerator           0
HelpfulnessDenominator         0
Time                           0
Score                     299996
dtype: int64

In [68]:
not_index = textfull.loc[textfull['Score'].notnull(), :].index
print(not_index)
empty = trainset.iloc[not_index]
#print(empty)
textfull1 = textfull.iloc[not_index]
X_train = textfull1['Text']
y_train = textfull1['Score']

Int64Index([      0,       1,       2,       3,       4,       6,       7,
                  8,       9,      10,
            ...
            1697492, 1697495, 1697497, 1697499, 1697501, 1697502, 1697505,
            1697506, 1697507, 1697508],
           dtype='int64', length=1397513)


In [69]:
print(textfull1.head())

   Unnamed: 0  Id                                               Text UserId  \
0           0   0  charm version classic dicken tale henri winkle...      1   
1           1   1  good emot move christma carol dicken like chri...      2   
2           2   2  dont get wrong winkler wonder charact actor wo...      3   
3           3   3  henri winkler good twist classic stori convent...      4   
4           4   4  one best scroog movi henri winkler outdo cast ...      5   

  ProductId  HelpfulnessNumerator  HelpfulnessDenominator        Time  Score  
0         1                     0                       0  1203984000    4.0  
1         1                     0                       0  1388361600    3.0  
2         1                     0                       0  1388361600    3.0  
3         1                     0                       0  1202860800    5.0  
4         1                     0                       0  1387670400    4.0  


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(X_train)
X = vectorizer.transform(X_train)

In [71]:
import numpy as np
from scipy.sparse import csr_matrix, vstack, hstack
add_csr = textfull1.loc[:, ('UserId', 'ProductId','HelpfulnessNumerator','HelpfulnessDenominator')].astype(int)
add_csr2 = add_csr.to_numpy()
text_add_csr = csr_matrix(add_csr2)

In [73]:
Xall = hstack((text_add_csr, X))

In [75]:
Xall = csr_matrix(Xall)

In [77]:
yall = y_train.to_numpy()

In [79]:
from sklearn.model_selection import KFold
from sklearn.svm import LinearSVC

In [80]:
def accuracy(predictions,y_true):
    return np.sum(predictions==y_true)/len(y_true)

**Linear regression**

In [82]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(Xall, yall)
reg.score(Xall, yall)

0.02006821882629406

In [112]:
from sklearn.linear_model import LinearRegression
kf = KFold(n_splits=5)
kf.get_n_splits(X)
print(kf)
KFold(n_splits=5, random_state=None, shuffle=False)
acc = 0
for train_index, test_index in kf.split(X):
    Xall_train, Xall_test = Xall[train_index], Xall[test_index]
    yall_train, yall_test = yall[train_index], yall[test_index]
    reg = LinearRegression().fit(Xall_train, yall_train)
    reg.score(Xall_train, yall_train)
    predictions=reg.predict(Xall_test)
    acc += accuracy(predictions,yall_test)
    print(acc)
print(acc/5)

KFold(n_splits=5, random_state=None, shuffle=False)
0.0
0.0
0.0
0.0
0.0
0.0


**naive_bayes_MultinomialNB**

In [31]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB().fit(Xall, yall)

**Ridge**

In [61]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=0.05, normalize=True).fit(Xall, yall)
print(ridge.score(Xall, yall))

0.7857460118595827


**Decision Tree**

In [ ]:
from sklearn import tree
dt = tree.DecisionTreeClassifier(random_state = 0, max_depth=20).fit(Xall, yall)

**Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_depth=20, random_state=0).fit(Xall, yall)

**SVM**

In [36]:
kf = KFold(n_splits=5)
kf.get_n_splits(X)
print(kf)

KFold(n_splits=5, random_state=None, shuffle=False)


In [38]:
KFold(n_splits=5, random_state=None, shuffle=False)
acc = 0
for train_index, test_index in kf.split(X):
    Xall_train, Xall_test = Xall[train_index], Xall[test_index]
    yall_train, yall_test = yall[train_index], yall[test_index]
    clf = LinearSVC(dual=False,penalty='l1')
    clf.fit(Xall_train, yall_train)
    predictions=clf.predict(Xall_test)
    acc += accuracy(predictions,yall_test)
    print(acc)
print(acc/5)

0.645832066203225
1.2452496037609615
1.8525990776485406
2.4708379453489506
3.0859569784864593
0.6171913956972919


# Real Predict

In [83]:
textfull2 = pd.read_csv('textfull.csv')
print(textfull2.shape)

(1697533, 3)


In [84]:
textfull2.head()

,Unnamed: 0,Id,Text
0,0,0,charm version classic dicken tale henri winkle...
1,1,1,good emot move christma carol dicken like chri...
2,2,2,dont get wrong winkler wonder charact actor wo...
3,3,3,henri winkler good twist classic stori convent...
4,4,4,one best scroog movi henri winkler outdo cast ...


In [85]:
textfull2=pd.concat([textfull2,trainset.loc[:, ('UserId', 'ProductId','HelpfulnessNumerator','HelpfulnessDenominator')]],axis=1)

In [86]:
textfull2.head()

,Unnamed: 0,Id,Text,UserId,ProductId,HelpfulnessNumerator,HelpfulnessDenominator,Time
0,0,0,charm version classic dicken tale henri winkle...,1,1,0,0,1203984000
1,1,1,good emot move christma carol dicken like chri...,2,1,0,0,1388361600
2,2,2,dont get wrong winkler wonder charact actor wo...,3,1,0,0,1388361600
3,3,3,henri winkler good twist classic stori convent...,4,1,0,0,1202860800
4,4,4,one best scroog movi henri winkler outdo cast ...,5,1,0,0,1387670400


In [87]:
null_index_trainset = trainset.loc[trainset['Score'].isnull(), :].index
print(null_index_trainset)
textfull_testall = textfull2.iloc[null_index_trainset]
print(textfull_testall.shape)
print(textfull_testall)

Int64Index([      5,      11,      17,      46,      47,      48,      55,
                 59,      78,      79,
            ...
            1697473, 1697493, 1697499, 1697517, 1697518, 1697520, 1697522,
            1697524, 1697527, 1697528],
           dtype='int64', length=300000)
(300000, 8)
         Unnamed: 0       Id  \
5                 5        5   
11               11       11   
17               17       17   
46               46       46   
47               47       47   
...             ...      ...   
1697520     1697520  1697520   
1697522     1697522  1697522   
1697524     1697524  1697524   
1697527     1697527  1697527   
1697528     1697528  1697528   

                                                      Text  UserId ProductId  \
5        favorit movi mine long time henri winkler grea...       6         1   
11       even though dont care henri winkler act like m...      12         1   
17       anorh good movi holiday watchersa littl twist ...      18         1 

In [88]:
textfull_testall.isna().sum()
null_testall = textfull_testall.loc[textfull_testall.loc[:, ('Text')].isnull(), :]
print(null_testall)

         Unnamed: 0       Id Text  UserId ProductId  HelpfulnessNumerator  \
805439       805439   805439  NaN   43970     19924                     0   
1233308     1233308  1233308  NaN   63603     36620                     0   
1575771     1575771  1575771  NaN  120440     46628                     0   
1621124     1621124  1621124  NaN  121698     47785                     0   

         HelpfulnessDenominator        Time  
805439                        1  1404950400  
1233308                       0  1405296000  
1575771                       0  1404259200  
1621124                       0  1404950400  


In [89]:
textfull_testall.dropna(subset = ['Text'],inplace = True)
textfull_testall.reset_index(inplace=True, drop=True)
textfull_testall.isna().sum()
textfull_testall.head()

<ipython-input-89-562f893f55d1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  textfull_testall.dropna(subset = ['Text'],inplace = True)


,Unnamed: 0,Id,Text,UserId,ProductId,HelpfulnessNumerator,HelpfulnessDenominator,Time
0,5,5,favorit movi mine long time henri winkler grea...,6,1,0,0,1383696000
1,11,11,even though dont care henri winkler act like m...,12,1,0,0,1390780800
2,17,17,anorh good movi holiday watchersa littl twist ...,18,1,0,0,1389657600
3,46,46,wife grew new hampshir version take place easi...,47,1,10,14,1132963200
4,47,47,first didnt like movi think minut act wasnt,48,1,0,1,1384905600


In [90]:
X_test = textfull_testall['Text']

In [91]:
X_vec_test = vectorizer.transform(X_test)

In [93]:
add_csr_test = textfull_testall.loc[:, ('UserId', 'ProductId','HelpfulnessNumerator','HelpfulnessDenominator')].astype(int)
add_csr2_test = add_csr_test.to_numpy()
test_add_csr = csr_matrix(add_csr2_test)

In [95]:
Xall_test = hstack((test_add_csr, X_vec_test))

**SVM Predict**

In [114]:
#SVM
#yprediction=clf.predict(Xall_test)

**MNB Predict**

In [46]:
#MNB
#yprediction=mnb.predict(Xall_test)

**Decision Tree Predict**

In [113]:
#Decision Tree
#yprediction = dt.predict(Xall_test)

**Random Forest Predict**

In [ ]:
#Random Forest
#yprediction = rfc.predict(Xall_test)

**Ridge Predict**

In [76]:
#Ridge
yprediction = ridge.predict(Xall_test)
for i in range(len(yprediction)):
    s = yprediction[i]
    if s < 0.5:
        yprediction[i] = 0
    else:
        if s < 1.5:
            yprediction[i] = 1
        else:
            if s < 2.5:
                yprediction[i] = 2
            else:
                if s < 3.5:
                    yprediction[i] = 3
                else:
                    if s < 4.5:
                        yprediction[i] = 4
                    else:
                        yprediction[i] = 5

**Linear Regression Predict**

In [97]:
#linear regression
yprediction=reg.predict(Xall_test)
for i in range(len(yprediction)):
    s = yprediction[i]
    if s < 0.5:
        yprediction[i] = 0
    else:
        if s < 1.5:
            yprediction[i] = 1
        else:
            if s < 2.5:
                yprediction[i] = 2
            else:
                if s < 3.5:
                    yprediction[i] = 3
                else:
                    if s < 4.5:
                        yprediction[i] = 4
                    else:
                        yprediction[i] = 5

In [99]:
ypredict = pd.DataFrame(yprediction)

In [103]:
submit=pd.concat([textfull_testall['Id'],ypredict],axis=1)

In [104]:
submit.columns=['Id','Score']

# Filling missing value using summary predict

In [105]:
summarysubmit = pd.read_csv('submit_summary.csv')

In [107]:
null_testall=pd.merge(null_testall.loc[:, ('Id')],summarysubmit, how='left',on='Id')
null_testall

,Id,Score
0,805439,5.0
1,1233308,5.0
2,1575771,5.0
3,1621124,5.0


In [108]:
submit1=pd.merge(submit,null_testall.loc[:, ('Id', 'Score')], how='outer',)

In [109]:
submit1 = submit1.sort_values(by='Id', ascending=True)

In [110]:
submit1.head()

,Id,Score
0,5,4.0
1,11,4.0
2,17,4.0
3,46,4.0
4,47,4.0


# Store the result

In [111]:
submit1.to_csv('submit_text10.csv',index=False)
